<a href="https://colab.research.google.com/github/Varsha-commuri/Projects/blob/main/monkeypox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import zipfile
import os
uploaded = files.upload()
zip_file_path = next(iter(uploaded))
extract_dir = "/content/Monkeypox_Skin_Image_Dataset"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
os.listdir(extract_dir)


Saving archive(2).zip to archive(2).zip


NotADirectoryError: [Errno 20] Not a directory: '/content/archive(2).zip/Monkeypox Skin Image Dataset'

In [ ]:
os.listdir('/content/Monkeypox_Skin_Image_Dataset/Monkeypox Skin Image Dataset')



['Chickenpox', 'Measles', 'Monkeypox', 'Normal']

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to the extracted dataset
dataset_dir = '/content/Monkeypox_Skin_Image_Dataset/Monkeypox Skin Image Dataset'

# Create an ImageDataGenerator with rescaling and validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load training data
train_data_gen = datagen.flow_from_directory(
    dataset_dir,  # Path to the dataset
    target_size=(224, 224),  # Resize all images to 224x224
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='training'  # Use this subset for training
)

# Load validation data
val_data_gen = datagen.flow_from_directory(
    dataset_dir,  # Path to the dataset
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='validation'  # Use this subset for validation
)

# Print the number of samples in the training and validation sets
print(f"Training images: {train_data_gen.samples}, Validation images: {val_data_gen.samples}")


Found 618 images belonging to 4 classes.
Found 152 images belonging to 4 classes.
Training images: 618, Validation images: 152


In [ ]:
import tensorflow as tf

# Build a simple CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')  # 4 classes: chickenpox, measles, normal, monkeypox
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Multi-class classification
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_data_gen,
    epochs=10,  # You can adjust the number of epochs based on your dataset
    validation_data=val_data_gen
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 91s 4s/step - accuracy: 0.3490 - loss: 2.5364 - val_accuracy: 0.5592 - val_loss: 1.1519
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 98s 5s/step - accuracy: 0.5844 - loss: 1.0849 - val_accuracy: 0.5921 - val_loss: 1.0110
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 133s 5s/step - accuracy: 0.6283 - loss: 1.0027 - val_accuracy: 0.5921 - val_loss: 1.0117
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - accuracy: 0.6642 - loss: 0.8821 - val_accuracy: 0.5724 - val_loss: 1.0628
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.7055 - loss: 0.7576 - val_accuracy: 0.5724 - val_loss: 1.1021
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - accuracy: 0.7656 - loss: 0.6603 - val_accuracy: 0.5724 - val_loss: 1.0852
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 100s 5s/step - accuracy: 0.8101 - loss: 0.5082 - val_accuracy: 0.5789 - val_loss: 1.1105
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 89s 4s/step - accuracy: 0.8422 - loss: 0.4111 - val_accuracy: 0.5658 - val_loss: 1.179

In [ ]:
# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(val_data_gen)
print(f"Validation accuracy: {val_acc:.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 940ms/step - accuracy: 0.5640 - loss: 1.6165
Validation accuracy: 0.5592


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation (in addition to rescaling)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Keep validation split
)

# Only rescaling for validation
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Create new train and validation generators with augmentation
train_data_gen = train_datagen.flow_from_directory(
    dataset_dir,  # Path to the dataset
    target_size=(224, 224),  # Resize all images to 224x224
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Use this subset for training
)

val_data_gen = val_datagen.flow_from_directory(
    dataset_dir,  # Path to the dataset
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use this subset for validation
)


Found 618 images belonging to 4 classes.
Found 152 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Load pre-trained VGG16 model without the top layer (include_top=False)
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Build the model by adding custom layers on top
model = models.Sequential([
    base_model,  # Pre-trained model
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # 4 classes: chickenpox, measles, normal, monkeypox
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_data_gen,
    epochs=10,
    validation_data=val_data_gen
)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 482s 24s/step - accuracy: 0.5274 - loss: 1.1788 - val_accuracy: 0.5789 - val_loss: 1.0421
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 481s 24s/step - accuracy: 0.5385 - loss: 1.0892 - val_accuracy: 0.5789 - val_loss: 0.9892
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 481s 24s/step - accuracy: 0.6057 - loss: 1.0090 - val_accuracy: 0.5855 - val_loss: 0.9600
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 498s 24s/step - accuracy: 0.6971 - loss: 0.8532 - val_accuracy: 0.6250 - val_loss: 0.8809
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 482s 24s/step - accuracy: 0.6665 - loss: 0.8267 - val_accuracy: 0.6316 - val_loss: 0.8469
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 497s 24s/step - accuracy: 0.6963 - loss: 0.7750 - val_accuracy: 0.6447 - val_loss: 0.8284
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 500s 24s/step - accuracy: 0.7008 - loss: 0.7679 - val_accuracy: 0.6711 - val_loss: 0.8222
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 476s 24s/step - accuracy: 0.6873 - loss: 0.7902 - val_accuracy: 0.6579 - val_

In [ ]:
# Evaluate the model on the validation data
val_loss, val_acc = model.evaluate(val_data_gen)

# Print the validation accuracy
print(f"Validation accuracy: {val_acc:.4f}")


NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras import layers, models
import tensorflow as tf

# Unfreeze some of the base model for fine-tuning
base_model.trainable = True

# Freeze the first few layers and unfreeze the last layers (you can adjust this number)
for layer in base_model.layers[:15]:  # Freeze the first 15 layers, adjust this as needed
    layer.trainable = False

# Recompile the model after unfreezing the layers (use a lower learning rate for fine-tuning)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Reduced learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Re-train the model (fine-tune)
history = model.fit(
    train_data_gen,
    epochs=10,  # Increase epochs if needed
    validation_data=val_data_gen
)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 609s 30s/step - accuracy: 0.7311 - loss: 0.6970 - val_accuracy: 0.7171 - val_loss: 0.7408
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 571s 28s/step - accuracy: 0.7594 - loss: 0.6328 - val_accuracy: 0.7237 - val_loss: 0.7162
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 561s 28s/step - accuracy: 0.7863 - loss: 0.5783 - val_accuracy: 0.7237 - val_loss: 0.6869
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 602s 30s/step - accuracy: 0.7816 - loss: 0.5773 - val_accuracy: 0.7566 - val_loss: 0.6655
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 551s 28s/step - accuracy: 0.7898 - loss: 0.5514 - val_accuracy: 0.7632 - val_loss: 0.6693
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 604s 30s/step - accuracy: 0.8380 - loss: 0.4372 - val_accuracy: 0.7566 - val_loss: 0.6539
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 555s 28s/step - accuracy: 0.8253 - loss: 0.4235 - val_accuracy: 0.7763 - val_loss: 0.6233
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 554s 28s/step - accuracy: 0.8483 - loss: 0.3875 - val_accuracy: 0.

In [ ]:
val_loss, val_acc = model.evaluate(val_data_gen)
print(f"Fine-tuned Validation accuracy: {val_acc:.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 93s 18s/step - accuracy: 0.7422 - loss: 0.6102
Fine-tuned Validation accuracy: 0.7566


In [ ]:
# Unfreeze more layers for fine-tuning
base_model.trainable = True

# Unfreeze all layers after block 5 (adjust the number of layers you want to unfreeze)
for layer in base_model.layers[:15]:  # Freeze the first 15 layers
    layer.trainable = False

# Recompile the model after making changes to the base model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Re-train the model
history = model.fit(
    train_data_gen,
    epochs=10,
    validation_data=val_data_gen
)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 610s 30s/step - accuracy: 0.8644 - loss: 0.3636 - val_accuracy: 0.7895 - val_loss: 0.6219
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 569s 28s/step - accuracy: 0.8781 - loss: 0.3353 - val_accuracy: 0.8092 - val_loss: 0.6045
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 554s 28s/step - accuracy: 0.8878 - loss: 0.3258 - val_accuracy: 0.8092 - val_loss: 0.6155
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 611s 30s/step - accuracy: 0.8855 - loss: 0.2998 - val_accuracy: 0.8158 - val_loss: 0.5985
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 603s 30s/step - accuracy: 0.8940 - loss: 0.2850 - val_accuracy: 0.7961 - val_loss: 0.6348
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 574s 28s/step - accuracy: 0.8998 - loss: 0.2872 - val_accuracy: 0.7961 - val_loss: 0.6218
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 612s 30s/step - accuracy: 0.9158 - loss: 0.2753 - val_accuracy: 0.7961 - val_loss: 0.5755
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 572s 28s/step - accuracy: 0.9235 - loss: 0.2495 - val_accuracy: 0.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,  # Increase the rotation range
    width_shift_range=0.4,  # Increased width shift
    height_shift_range=0.4,  # Increased height shift
    shear_range=0.4,  # Increased shear
    zoom_range=0.4,  # Increased zoom range
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Keep validation split
)
